# Imports

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from asapdiscovery.docking.analysis import calculate_rmsd_openeye
from asapdiscovery.data.schema.ligand import Ligand
from asapdiscovery.data.readers.molfile import MolFileFactory
from asapdiscovery.modeling.protein_prep import ProteinPrepper
import plotly.express as px

# Load Data

In [ ]:
# results_path = Path("/lila/data/chodera/asap-datasets/retro_docking/sars_fragalysis_retrospective/20240424_multi_pose_docking_cross_docking/")
results_path = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20240424_multi_pose_docking_cross_docking/results_csvs")

In [ ]:
# cache_path = Path("/data/chodera/asap-datasets/mpro_fragalysis-04-01-24_curated_cache")
cache_path = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/mpro_fragalysis-04-01-24_curated_cache")

In [ ]:
# original_ligand_path = Path("/data/chodera/asap-datasets/mpro_fragalysis-04-01-24_curated_cache/combined_3d.sdf")
original_ligand_path = cache_path / "combined_3d.sdf"

In [ ]:
results_path.exists()

In [ ]:
original_ligand_path.exists()

In [ ]:
from asapdiscovery.modeling.protein_prep import ProteinPrepper
complexes = ProteinPrepper.load_cache(cache_path)

In [ ]:
cmpd_to_frag_dict = {c.ligand.compound_name: c.target.target_name for c in complexes}

In [ ]:
mff = MolFileFactory(filename=original_ligand_path)
ligs = mff.load()

In [ ]:
lig_dict = {lig.compound_name: lig for lig in ligs}

In [ ]:
results_paths = list(results_path.glob("20240430*.csv"))

In [ ]:
dfs = [pd.read_csv(_csv) for _csv in results_paths]

In [ ]:
df = pd.concat(dfs)

In [ ]:
len(df)

# Basic Analysis

## What is the distribution of targets included in each ligand

In [ ]:
target_distribution = df.groupby(["Query_Ligand"]).nunique()

In [ ]:
fig = px.histogram(target_distribution, x="Reference_Ligand")
fig.write_image("target_distribution.png")

# What is the distribution of ligands successfully docked per target?

In [ ]:
ligand_distribution = df.groupby(["Reference_Ligand"]).nunique()

In [ ]:
fig = px.histogram(ligand_distribution, x="Query_Ligand")
fig.write_image("ligand_distribution.png")

## are all targets and ligands represented?

In [ ]:
original_lig_ids = set(lig_dict.keys())

In [ ]:
original_lig_ids - set(df.Query_Ligand)

In [ ]:
original_lig_ids - set(df.Reference_Ligand)

In [ ]:
set(df.Query_Ligand) - set(df.Reference_Ligand)

In [ ]:
set(df.Reference_Ligand) - set(df.Query_Ligand)

### so 3 structures are never docked to:
'BRU-THA-92256091-17', 'PET-UNK-8df914d1-2', 'RAL-THA-8416115c-13'

In [ ]:
missing_refs = [lig_dict[missing] for missing in original_lig_ids - set(df.Reference_Ligand)]

## what pairs are present?

In [ ]:
refs = df.Reference_Ligand
queries = df.Query_Ligand
pairs = {(ref, query) for ref, query in zip(refs,queries)}

In [ ]:
len(pairs)

In [ ]:
('MAT-POS-4223bc15-18', 'MAT-POS-2e8b2191-10') in  pairs

In [ ]:
from itertools import permutations
possible_pairs = set(list(permutations(lig_dict.keys(), 2)))

In [ ]:
len(possible_pairs)

# Adding null results

In [ ]:
missing_pairs = possible_pairs - pairs

In [ ]:
len(missing_pairs)

In [ ]:
null_df = pd.DataFrame({"Reference_Ligand": [i for i,j in missing_pairs], 
                        "Query_Ligand":[j for i,j in missing_pairs],
                       "RMSD": np.nan,
                       "Pose_ID": 0,
                       "POSIT_Method": "Failed"})

In [ ]:
null_df

## add structures

In [ ]:
null_df["Reference_Structure"] = null_df.Reference_Ligand.apply(lambda x: cmpd_to_frag_dict[x])

## concat null results

In [ ]:
padded = pd.concat([df, null_df])

In [ ]:
df = padded.copy()
df = df.reindex()

In [ ]:
df[(df["Reference_Ligand"] == "MAT-POS-4223bc15-18")&(df["Query_Ligand"] == "MAT-POS-2e8b2191-10")]

## check that all pairs are present now

In [ ]:
refs = df.Reference_Ligand
queries = df.Query_Ligand
pairs = {(ref, query) for ref, query in zip(refs,queries)}

In [ ]:
len(pairs) == len(possible_pairs)

# Add Query_Structure 

## make sure the references structures match?

In [ ]:
all(df["Reference_Structure"] == df.Reference_Ligand.apply(lambda x: cmpd_to_frag_dict[x]))

## they do, good

In [ ]:
df["Query_Structure"] = df.Query_Ligand.apply(lambda x: cmpd_to_frag_dict[x])

# what is the RMSD distribution?

In [ ]:
fig = px.histogram(df, x="RMSD")
fig.write_image("rmsd_distribution.png")

In [ ]:
failed_rmsd_calculation = df[df.RMSD == -1]

In [ ]:
failed_rmsd_calculation.nunique()

In [ ]:
failed_rmsd_calculation.Query_Ligand.unique()

In [ ]:
failed_rmsd_calculation.Query_Structure.unique()

## all the failed rmsd calculations are from 'MAT-POS-5d65ec79-1' - which I have now fixed

## lets just set those to null for now

In [ ]:
# df["RMSD"] = df.RMSD.apply(lambda x: np.nan if x == -1 else x)

# Save the combined csv

# save a subset for testing

In [ ]:
df.to_csv(results_path / "20240502_combined_results_with_rmsd_and_null_results.csv")

In [ ]:
randomized = df.sample(frac=0.001, replace=False)

In [ ]:
randomized.RMSD.min()

In [ ]:
randomized.RMSD.max()

In [ ]:
randomized.nunique()

In [ ]:
randomized.to_csv(results_path / "20240502_combined_results_with_rmsd_and_null_results_subset.csv")

# Add Date Information

# Add Chemical Similarity Information

## Num Atoms in MCSS

In [ ]:
num_atoms_in_mcss = pd.read_csv("20240503_ligand_analysis/20240503_mcss_num_atoms.csv")

In [ ]:
df = df.merge(num_atoms_in_mcss, on=["Query_Ligand", "Reference_Ligand"], how="left")

## Tanimoto Similarity of ECFP4

In [ ]:
ecfp4_similarities = pd.read_csv("20240503_ligand_analysis/20240503_all_tc_comparison.csv")

In [ ]:
ecfp4_similarities = ecfp4_similarities[ecfp4_similarities["Fingerprint"] == "ECFP4"]

In [ ]:
df = df.merge(ecfp4_similarities, on=["Query_Ligand", "Reference_Ligand"], how="left")

## TanimotoCombo

# Save CSV with Data

In [ ]:
df.to_csv(results_path / "20240503_combined_results_with_data.csv")